In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

# Function to extract Product Title
def get_title(soup):
    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id": 'productTitle'})
        # Inner NavigatableString Object
        title_value = title.text
        # Title as a string value
        title_string = title_value.strip()
    except AttributeError:
        title_string = ""
    return title_string

# Function to extract Product Price
def get_price(soup):
    try:
        price = soup.find('span' , attrs={'class' : 'a-price aok-align-center'}).find('span' , attrs={'class' : 'a-offscreen'}).string.strip()
    except AttributeError:
        price = ""
    return price

if __name__ == '__main__':
    # Add your user agent
    HEADERS = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36 Edg/126.0.0.0',
        'Accept-Language': 'en-US, en;q=0.5'
    }

    # The webpage URL
    URL = "https://www.amazon.in/s?k=formal+shoes+for+men&crid=31MBP7IYXEE53&sprefix=for%2Caps%2C315&ref=nb_sb_ss_w_hit-vc-lth_formal-shoes-for-men_k0_1_3"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)
    webpage.raise_for_status()  # Raise an exception for HTTP errors

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class': 'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})

    # Store the links
    links_list = ["https://www.amazon.in" + link.get('href') for link in links]

    d = {"title": [], "price": [], "link": []}

    # Loop for extracting product details from each link
    for link in links_list:
        try:
            new_webpage = requests.get(link, headers=HEADERS)
            new_webpage.raise_for_status()  # Raise an exception for HTTP errors

            new_soup = BeautifulSoup(new_webpage.content, "html.parser")

            # Function calls to display all necessary product information
            d['title'].append(get_title(new_soup))
            d['price'].append(get_price(new_soup))
            d['link'].append(link)  # Append the link directly

        except requests.exceptions.RequestException as e:
            print(f"Failed to retrieve page: {link}\nError: {e}")
            d['title'].append("")
            d['price'].append("")
            d['link'].append("")

    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)


In [6]:
amazon_df

,title,price,link
0,Blue Leather mens 3943 Loafers|Slip on Formal ...,"₹1,499.00",https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
1,NOBILITY by Action Formal Slip On Shoe's for M...,"₹1,049.00",https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
2,ARAMISH Men's Genuine Leather Lace Up Formal S...,"₹1,699.00",https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
3,NOBILITY by Action Formal Slip On Shoes for Me...,"₹1,049.00",https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
4,Bata Men's MAX Formal,₹878.00,https://www.amazon.in/BATA-Mens-Black-Sneaker-...
...,...,...,...
61,FAUSTO Men's Formal Lace Up Oxford Shoes for O...,"₹1,216.00",https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
62,FAUSTO Men's Formal Office Dress Lace Up Derby...,₹831.00,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
63,Amazon Brand - Symbol Mens FlorenceFormal Shoes,₹689.00,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
64,Blue Leather mens 3908 Classic Slipon Formal S...,₹999.00,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
